In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.insert(1, '../Config')
from lablib import *

# Prima parte RC RL 

# CIRCUITO RC
 Capacità utilizzata= 99 nF +-1 \
 Resistenza= $69.5\pm0.1 k\Omega$